# autocorrelogram
Ref: Content‑based image retrieval using combined texture and color features based on multi‑resolution multi‑direction filtering and color autocorrelogram
https://doi.org/10.1007/s12652-019-01466-0

In [1]:
# %pip install numpy pillow

In [2]:
import numpy as np
from PIL import Image

In [3]:
# Computes neighbor counts for multiple pixels at a given distance.
# distance (int): The order of the neighborhood (inf-norm distance).
# pixel_widths, pixel_heights (ndarray): Arrays of pixel coordinates (width and height indices).
# target_colors (ndarray): Array of quantized colors to compare.
# quantized_image (ndarray): The color-quantized image matrix.
# image_width, image_height (int): Dimensions of the input image.
# same_color_counts (ndarray): Number of neighboring pixels with the same color for each pixel.
# valid_neighbor_counts (ndarray): Total number of valid neighboring pixels for each pixel.
def count_neighbors_at_distance(distance, pixel_widths, pixel_heights, target_colors, quantized_image, image_width, image_height):
    num_pixels = len(pixel_widths)
    
    # Generate neighbor offsets for inf-norm distance (8*distance points)
    offsets = []
    for d in range(-distance, distance + 1):
        if abs(d) == distance:
            for w in range(-distance, distance + 1):
                offsets.append((w, d))
        else:
            offsets.append((-distance, d))
            offsets.append((distance, d))
    offsets = np.array(offsets)  # Shape: (8*distance, 2)

    # Expand offsets for all pixels
    offsets = offsets[:, None, :]  # Shape: (8*distance, 1, 2)
    pixel_coords = np.stack((pixel_widths, pixel_heights), axis=-1)  # Shape: (num_pixels, 2)
    pixel_coords = pixel_coords[None, :, :]  # Shape: (1, num_pixels, 2)
    
    # Compute neighbor coordinates for all pixels
    neighbor_coords = pixel_coords + offsets  # Shape: (8*distance, num_pixels, 2)
    neighbor_width_coords = neighbor_coords[:, :, 0]  # Shape: (8*distance, num_pixels)
    neighbor_height_coords = neighbor_coords[:, :, 1]  # Shape: (8*distance, num_pixels)

    # Check validity of neighbors (within image bounds)
    valid_mask = (
        (neighbor_width_coords > 0) & (neighbor_width_coords <= image_width) &
        (neighbor_height_coords > 0) & (neighbor_height_coords <= image_height)
    )  # Shape: (8*distance, num_pixels)

    # Initialize counts
    same_color_counts = np.zeros(num_pixels, dtype=np.int32)
    valid_neighbor_counts = np.zeros(num_pixels, dtype=np.int32)

    # Process each neighbor offset
    for i in range(8 * distance):
        valid_pixels = valid_mask[i]  # Shape: (num_pixels,)
        if not np.any(valid_pixels):
            continue
        
        # Get valid neighbor coordinates for this offset
        valid_widths = neighbor_width_coords[i, valid_pixels].astype(int) - 1
        valid_heights = neighbor_height_coords[i, valid_pixels].astype(int) - 1
        
        # Extract neighbor colors
        neighbor_colors = quantized_image[valid_heights, valid_widths]
        
        # Compare with target colors for valid pixels
        valid_target_colors = target_colors[valid_pixels]
        same_color_counts[valid_pixels] += (neighbor_colors == valid_target_colors).astype(np.int32)
        
        # Count valid neighbors
        valid_neighbor_counts[valid_pixels] += 1

    return same_color_counts, valid_neighbor_counts

In [4]:
# Creates the auto-correlogram vector for an input image efficiently.
# img (PIL.Image): Color image (RGB format).
# neighbor_distance_vector (list or ndarray): Vector of neighbor distances for color distribution calculation.
# color_quantization_number (int): Number of colors to quantize the image to (default: 64).
# correlogram_vector (ndarray): Vector of probabilities of occurrence of quantized colors.
def color_auto_correlogram(img, neighbor_distance_vector=[1, 3], color_quantization_number=64):
    # Quantize image
    img_quantized = img.quantize(colors=color_quantization_number, method=Image.Quantize.MEDIANCUT)
    quantized_image = np.array(img_quantized)  # Shape: (height, width)

    image_height, image_width = quantized_image.shape
    neighbor_dimension = len(neighbor_distance_vector)

    # Initialize count arrays
    same_color_counts = np.zeros((color_quantization_number, neighbor_dimension), dtype=np.float32)
    valid_neighbor_counts = np.zeros((color_quantization_number, neighbor_dimension), dtype=np.float32)

    # Generate all pixel coordinates
    pixel_widths, pixel_heights = np.meshgrid(
        np.arange(image_width), np.arange(image_height), indexing='xy'
    )
    pixel_widths = pixel_widths.ravel()  # Shape: (height * width,)
    pixel_heights = pixel_heights.ravel()  # Shape: (height * width,)
    target_colors = quantized_image[pixel_heights, pixel_widths]  # Shape: (height * width,)

    # Process each distance
    for distance_index, distance in enumerate(neighbor_distance_vector):
        # Compute neighbor counts for all pixels
        pixel_same_color_counts, pixel_valid_neighbor_counts = count_neighbors_at_distance(
            distance,
            pixel_widths,
            pixel_heights,
            target_colors,
            quantized_image,
            image_width,
            image_height
        )

        # Aggregate counts by color
        for color in range(color_quantization_number):
            color_mask = (target_colors == color)
            same_color_counts[color, distance_index] = np.sum(pixel_same_color_counts[color_mask])
            valid_neighbor_counts[color, distance_index] = np.sum(pixel_valid_neighbor_counts[color_mask])

    # Compute probabilities
    probability_distributions = same_color_counts / (1 + valid_neighbor_counts)  # Shape: (color_quantization_number, neighbor_dimension)

    # Flatten to correlogram vector
    correlogram_vector = probability_distributions.ravel()  # Shape: (color_quantization_number * neighbor_dimension,)

    return correlogram_vector

In [5]:
img = Image.open('peppers.png').convert('RGB')
correlogram_vector = color_auto_correlogram(img,
                                            neighbor_distance_vector = [1,2],
                                            color_quantization_number=8)
print(correlogram_vector.shape)
# Should be (96,) 95 = 64*3 i.e. color_quantization_number * neighbor_distance_vector


(16,)
